**Notebook for verification of 1-loop results**

In [3]:
import numpy as np
import sympy as sym
sym.init_printing()
from IPython.display import display_latex

#Initialise Diagram
externals = [sym.Symbol("p1"), sym.Symbol("p2"), sym.Symbol("p3"), sym.Symbol("p4")]
internals = [sym.Symbol("q1"), sym.Symbol("q2"), sym.Symbol("q3"), sym.Symbol("q4")]
externalmasses = [sym.Symbol("m1"), sym.Symbol("m2"), sym.Symbol("m3"), sym.Symbol("m4")]
s = sym.Symbol("s")
t = sym.Symbol("t")

display_latex(externals)
display_latex(externals)

In [31]:
from sympy.vector import Dot

def momentumConsConstraints(externals, internals):
    constraintEqns = [sym.Eq(internals[0] - internals[-1], externals[0])]
    for i in range(len(externals) - 1):
        constraintEqns.append(sym.Eq(internals[i+1] - internals[i], externals[i+1]))

    return constraintEqns

constraints = momentumConsConstraints(externals, internals)

display_latex(constraints)

def fourDot(vector1, vector2):
    #if vector1 == vector2:
    #    return vector1 ** 2
    #else:
        
    if vector1 == vector2 and vector1 == 0:
        return 0
    else:
        return Dot(vector1, vector2)

def getGramDet(internals):
    #Choose to cut always the last momentum
    cutInternals = internals[:-1]

    tempGramMatrix = []
    temp = []

    for i in range(len(cutInternals)):
        for j in range(len(cutInternals)):
            temp.append(fourDot(cutInternals[i] - internals[-1], cutInternals[j] - internals[-1]))
            if j == len(cutInternals) - 1:
                tempGramMatrix.append(temp)
                temp = []

    gramMatrix = sym.Matrix(tempGramMatrix)

    return gramMatrix, gramMatrix.det()

def getCayleyDet(internals, squaredMasses = None):
    if squaredMasses == None:
        tempCayley = []
        temp = []

        for i in range(len(internals)):
            for j in range(len(internals)):
                temp.append(1/2 * (-fourDot(internals[i] - internals[j], internals[i] - internals[j])))
                if j == len(internals) - 1:
                    tempCayley.append(temp)
                    temp = []
    else:
        tempCayley = []
        temp = []

        for i in range(len(internals)):
            for j in range(len(internals)):
                temp.append(1/2 * (-fourDot(internals[i] - internals[j], internals[i] - internals[j]) + squaredMasses[i] + squaredMasses[j]))
                if j == len(internals) - 1:
                    tempCayley.append(temp)
                    temp = []

    cayleyMatrix = sym.Matrix(tempCayley)

    display_latex(cayleyMatrix)

    return cayleyMatrix.det()

display_latex(getGramDet(internals))

def getMandelstams(externals):

    sEqn = sym.Eq((externals[0] + externals[1]) ** 2, sym.Symbol("s"))
    tEqn = sym.Eq((externals[0] + externals[3]) ** 2, sym.Symbol("t"))
    uEqn = sym.Eq((externals[0] + externals[2]) ** 2, sym.Symbol("u"))

    sEqn2 = sym.Eq(Dot(externals[0] + externals[1], externals[0] + externals[1]), sym.Symbol("s"))
    tEqn2 = sym.Eq(Dot(externals[0] + externals[3], externals[0] + externals[3]), sym.Symbol("t"))
    uEqn2 = sym.Eq(Dot(externals[0] + externals[2], externals[0] + externals[2]), sym.Symbol("u"))

    return sEqn, tEqn, uEqn, sEqn2, tEqn2, uEqn2


def subForExternals(internalGramDet, internals, externals):

    constraintEqns = momentumConsConstraints(externals, internals)
    externalGramDet = internalGramDet

    #This handles the trivial substitutions q_i - q_(i-1) = p_i
    for constraint in constraintEqns:
        externalGramDet = externalGramDet.subs(constraint.lhs, constraint.rhs)

    #This handles other conditions of the form q_i - q_j = F({p_i})
    for i in range(len(internals)):
        for j in range(len(internals)):

            currentEqn = sym.Eq(constraintEqns[i].lhs + constraintEqns[j].lhs, constraintEqns[i].rhs + constraintEqns[j].rhs)

            externalGramDet = externalGramDet.subs(currentEqn.lhs, currentEqn.rhs)

    return externalGramDet

def finishDotProdsSquare(expression, externals, externalMasses):

    for i in range(len(externals)):
        #Handle trivial p^2 substitutions
        expression = expression.subs(Dot(externals[i],externals[i]), externals[i] ** 2)
        expression = expression.subs(Dot(-externals[i],-externals[i]), externals[i] ** 2)
        expression = expression.subs(externals[i] ** 2, externalMasses[i] ** 2)

        #Handles sums of p_i's 
        for j in range(len(externals)):
            sumij = externals[i] + externals[j]
            minussumij = -externals[i] + -externals[j]

            expression = expression.subs(Dot(minussumij, minussumij), sumij ** 2)
            expression = expression.subs(Dot(sumij, sumij), sumij ** 2)

    #The above gets us into a form that's almost there, but with some remaining p_i dot p_j terms

    s,u,t,s2,u2,t2 = getMandelstams(externals)

    expression = expression.subs(s.lhs, s.rhs)
    expression = expression.subs(u.lhs, u.rhs)
    expression = expression.subs(t.lhs, t.rhs)

    expression = expression.subs(s2.lhs, s2.rhs)
    expression = expression.subs(u2.lhs, u2.rhs)
    expression = expression.subs(t2.lhs, t2.rhs)

    #Handle p_i dot p_j by subbing for Mandelstam invariants

    #expression = expression.subs(externals[0] + externals[1], -externals[2] - externals[3])

    expression = sym.simplify(expression)

    expression = expression.subs(Dot(externals[0], externals[1]), 1/2 * (sym.Symbol("s") - externals[0] ** 2 - externals[1] ** 2))
    expression = expression.subs(Dot(externals[0], -externals[3]), -1/2 * (sym.Symbol("u") - externals[0] ** 2 - externals[3] ** 2))
    expression = expression.subs(Dot(-externals[3], externals[1]), -1/2 * (sym.Symbol("t") - externals[1] ** 2 - externals[3] ** 2))

    # expression = expression.subs(Dot(externals[0], -externals[2]), -1/2 * (sym.Symbol("u") - externals[0] ** 2 - externals[2] ** 2))
    # expression = expression.subs(Dot(-externals[2], -externals[3]), 1/2 * (sym.Symbol("s") - externals[2] ** 2 - externals[3] ** 2))

    #Commented substitutions are unnecessary

    for i in range(len(externals)):
        #Handle trivial p^2 substitutions again (order matters)
        expression = expression.subs(Dot(externals[i],externals[i]), externals[i] ** 2)
        expression = expression.subs(Dot(-externals[i],-externals[i]), externals[i] ** 2)
        expression = expression.subs(externals[i] ** 2, externalMasses[i] ** 2)

    return expression

print("Gram Matrix:")

gramMatrix, gramDet = getGramDet(internals)

gramMatrix = subForExternals(gramMatrix, internals, externals)

print("With externals subbed in:")

display_latex(gramMatrix)

gramMatrix = finishDotProdsSquare(gramMatrix, externals, externalmasses)

print("With Mandelstam variables subbed in:")

display_latex(gramMatrix)

gramDet = subForExternals(gramDet, internals, externals)

print("Gram Determinant")

scaledGramDet = 4 * sym.expand(finishDotProdsSquare(gramDet, externals, externalmasses))

display_latex(scaledGramDet)

s = sym.Symbol("s")
t = sym.Symbol("t")
u = sym.Symbol("u")

scaledGramDet = scaledGramDet.subs(u, (externalmasses[0] ** 2 + externalmasses[1] ** 2 + externalmasses[2] ** 2 + externalmasses[3] ** 2 - s - t))

print("Gram Determinant (with u eliminated for ease of comparison with PLD.jl output)")

display_latex(sym.simplify(sym.expand(scaledGramDet)))

print("Cayley Matrix:")

cayleyDet = getCayleyDet(internals)

cayleyDet = subForExternals(cayleyDet, internals, externals)

print("Cayley Determinant")

display_latex(16 * finishDotProdsSquare(cayleyDet, externals, externalmasses))



#WORKING FOR SQUARE WITH MASSLESS INTERIOR (consistent with PLD.jl output)


Gram Matrix:
With externals subbed in:


With Mandelstam variables subbed in:


Gram Determinant


Gram Determinant (with u eliminated for ease of comparison with PLD.jl output)


Cayley Matrix:


Cayley Determinant


In [64]:
def getSijs(externals):
    diagramSize = len(externals)
    Sijs =  [sym.Eq(Dot(externals[0] + externals[-1], externals[0] + externals[-1]), sym.Symbol("s1" + str(diagramSize))), \
             sym.Eq(Dot(-externals[0] - externals[-1], -externals[0] - externals[-1]), sym.Symbol("s1" + str(diagramSize)))]
    for i in range(len(externals) -1):
        Sijs.append(sym.Eq((externals[i] + externals[i+1]) ** 2, sym.Symbol("s" + str(i+1) + str(i+2))))
        Sijs.append(sym.Eq(Dot(externals[i] + externals[i+1],externals[i] + externals[i+1]), sym.Symbol("s" + str(i+1) + str(i+2))))
        Sijs.append(sym.Eq(Dot(-externals[i] - externals[i+1], -externals[i] - externals[i+1]), sym.Symbol("s" + str(i+1) + str(i+2))))

    display_latex(Sijs)

    return Sijs

def getSijsSubs(externals):
    diagramSize = len(externals)

    SijConditions = []
    for i in range(diagramSize):
        for j in range(diagramSize):
            SijConditions.append(sym.Eq(Dot(externals[i], externals[j]), 1/2 * \
                                         (sym.Symbol("s" + str((i % diagramSize) + 1) + str((j % diagramSize) + 1)) - externals[i] ** 2 - externals[j] ** 2)))
            SijConditions.append(sym.Eq(Dot(-externals[i], -externals[j]), 1/2 * \
                                         (sym.Symbol("s" + str((i % diagramSize) + 1) + str((j % diagramSize) + 1)) - externals[i] ** 2 - externals[j] ** 2)))
            SijConditions.append(sym.Eq(Dot(-externals[i], externals[j]), -1/2 * \
                                         (sym.Symbol("s" + str((i % diagramSize) + 1) + str((j % diagramSize) + 1)) - externals[i] ** 2 - externals[j] ** 2)))
            SijConditions.append(sym.Eq(Dot(externals[i], -externals[j]), -1/2 * \
                                         (sym.Symbol("s" + str((i % diagramSize) + 1) + str((j % diagramSize) + 1)) - externals[i] ** 2 - externals[j] ** 2)))
            
    return SijConditions
            
def getRedundantSijs(externals):
    return None


def finishDotProds(expression, externals, externalMasses):

    for i in range(len(externals)):
        #Handle trivial p^2 substitutions
        expression = expression.subs(Dot(externals[i],externals[i]), externals[i] ** 2)
        expression = expression.subs(Dot(-externals[i],-externals[i]), externals[i] ** 2)
        expression = expression.subs(externals[i] ** 2, externalMasses[i] ** 2)

        #Handles sums of p_i's 
        for j in range(len(externals)):
            sumij = externals[i] + externals[j]
            minussumij = -externals[i] + -externals[j]

            expression = expression.subs(Dot(minussumij, minussumij), sumij ** 2)
            expression = expression.subs(Dot(sumij, sumij), sumij ** 2)

    #The above gets us into a form that's almost there, but with some remaining p_i dot p_j terms

    for i in range(len(externals)):
        #Handle trivial p^2 substitutions again (order matters)
        expression = expression.subs(Dot(externals[i],externals[i]), externals[i] ** 2)
        expression = expression.subs(Dot(-externals[i],-externals[i]), externals[i] ** 2)
        expression = expression.subs(externals[i] ** 2, externalMasses[i] ** 2)

    #Sub for Sijs directly where possible
    Sijs = getSijs(externals)

    for condition in Sijs:
        expression = expression.subs(condition.lhs, condition.rhs)

    expression = sym.simplify(expression)

    for i in range(len(externals)):
        #Handle trivial p^2 substitutions yet again
        expression = expression.subs(Dot(externals[i],externals[i]), externals[i] ** 2)
        expression = expression.subs(Dot(-externals[i],-externals[i]), externals[i] ** 2)
        expression = expression.subs(externals[i] ** 2, externalMasses[i] ** 2)

    SijsConditions = getSijsSubs(externals)

    for condition in SijsConditions:
        expression = expression.subs(condition.lhs, condition.rhs)

    return expression

#Initialise Diagram
externals = [sym.Symbol("p1"), sym.Symbol("p2"), sym.Symbol("p3"), sym.Symbol("p4"), sym.Symbol("p5")]
internals = [sym.Symbol("q1"), sym.Symbol("q2"), sym.Symbol("q3"), sym.Symbol("q4"), sym.Symbol("q5")]
externalmasses = [sym.Symbol("m1"), sym.Symbol("m2"), sym.Symbol("m3"), sym.Symbol("m4"), sym.Symbol("m5")]

print("Gram Matrix:")

gramMatrix, gramDet = getGramDet(internals)

display_latex(gramMatrix)

gramMatrix = subForExternals(gramMatrix, internals, externals)

print("With externals subbed in:")

display_latex(gramMatrix)

display_latex(finishDotProds(finishDotProds(gramMatrix, externals, externalmasses), externals, externalmasses))

testexpr = sym.Symbol("s15") + externals[3] ** 2 - externals[4] ** 2 - 2 * (Dot(externals[1] + externals[2], externals[3] + externals[4]) + sym.Symbol("s45"))
display_latex(finishDotProds(testexpr, externals, externalmasses))


Gram Matrix:


With externals subbed in:
